In [33]:
!pip install --user plotly panel numpy


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

Read transaction data with categories

In [35]:
# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('transactions_2022_2023_categorized.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])
df

,Date,Name / Description,Expense/Income,Amount (CAD),Category,Year,Month,Month Name
0,2023-12-30,Belastingdienst,Expense,9.96,Taxes,2023,12,Dec
1,2023-12-30,Tesco Breda,Expense,17.53,Groceries,2023,12,Dec
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,Housing,2023,12,Dec
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,Shopping,2023,12,Dec
4,2023-12-29,Selling Paintings,Income,13.63,Sales,2023,12,Dec
5,2023-12-29,Spotify Ab By Adyen,Expense,12.19,Entertainment,2023,12,Dec
6,2023-12-23,Tk Maxx Amsterdam Da,Expense,27.08,NaN,2023,12,Dec
7,2023-12-22,Consulting,Income,541.57,Work,2023,12,Dec
8,2023-12-22,Aidsfonds,Expense,10.70,Charity,2023,12,Dec
9,2023-12-20,Consulting,Income,2641.93,Work,2023,12,Dec


In [36]:
# For Income rows, assign Name / Description to Category
df['Category'] = np.where(df['Expense/Income'] == 'Income', df['Name / Description'], df['Category'])

In [37]:
def make_pie_chart(df, year, label):
    # Filter the dataset for expense transactions
    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount (CAD)', names='Category', color_discrete_sequence = color_scale)
    pie_fig.update_traces(textposition='inside', direction ='clockwise', hole=0.3, textinfo="label+percent")

    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount (CAD)'].sum() 
    total_income = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]['Amount (CAD)'].sum()
    
    if label == 'Expense':
        total_text = "€ " + str(round(total_expense))

        # Saving rate:
        saving_rate = round((total_income - total_expense)/total_income*100)
        saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
    else:
        saving_rate_text = ""
        total_text = "€ " + str(round(total_income))

    pie_fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',
                        title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text, 
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    return pie_fig

In [40]:
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
income_pie_fig_2022

print(income_pie_fig_2022)

Figure({
    'data': [{'direction': 'clockwise',
              'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hole': 0.3,
              'hovertemplate': 'Category=%{label}<br>Amount (CAD)=%{value}<extra></extra>',
              'labels': array(['Salary', 'Freelancing', 'Blogging'], dtype=object),
              'legendgroup': '',
              'name': '',
              'showlegend': True,
              'textinfo': 'label+percent',
              'textposition': 'inside',
              'type': 'pie',
              'values': array([  14.36, 2409.55, 4044.27])}],
    'layout': {'annotations': [{'font': {'size': 12}, 'showarrow': False, 'text': '€ 6468', 'x': 0.5, 'y': 0.5}],
               'legend': {'tracegroupgap': 0},
               'margin': {'t': 60},
               'piecolorway': [rgb(102,194,165), rgb(252,141,98),
                               rgb(141,160,203), rgb(231,138,195), rgb(166,216,84),
                               rgb(255,217,47), rgb(229,196,148), rgb(179,

Make bar charts over months in a year

In [44]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (CAD)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (CAD)', text_auto='.2s', title=label+" per month", color='Amount (CAD)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

In [49]:
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
income_monthly_2022

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'Month Name=%{x}<br>Amount (CAD)=%{marker.color}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': array([4044.27, 2423.91]), 'coloraxis': 'coloraxis', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'texttemplate': '%{y:.2s}',
              'type': 'bar',
              'x': array(['Oct', 'Nov'], dtype=object),
              'xaxis': 'x',
              'y': array([4044.27, 2423.91]),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'coloraxis': {'colorbar': {'title': {'text': 'Amount (CAD)'}},
                             'colorscale': [[0.0, 'rgb(255,255,229)'], [0.125,
                                            'rgb(247,252,185)'], [0.25,
                                            'rgb(217,240,163)'], [0.375,
                                            'rgb(173,221,142)'], [0.5,
                                            'rgb(120,198,121)'], [0.625,
                                            'rgb(65,171,93)'], [0.75,
                                            'rgb(35,132,67)'], [0.875,
                                            'rgb(0,104,55)'], [1.0,
                                            'rgb(0,69,41)']]},
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Income per month'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Month Name'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Amount (CAD)'}}}
})

Putting all charts together into tabs for 2022/2023

In [50]:
# Pie charts
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
expense_pie_fig_2022 = make_pie_chart(df, 2022, 'Expense')  
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')

# Bar charts
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
expense_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Expense')
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')

# Create tabs
tabs = pn.Tabs(
                        ('2022', pn.Column(pn.Row(income_pie_fig_2022, expense_pie_fig_2022),
                                                pn.Row(income_monthly_2022, expense_monthly_2022))),
                        ('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                pn.Row(income_monthly_2023, expense_monthly_2023))
                        )
                )
tabs.show()

Launching server at http://localhost:49660


Create dashboard

In [47]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"), 
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()

Launching server at http://localhost:49601
